# Implement Multi Agent Framework

In [21]:
import sys
sys.path.append('../src')

import os
from openai import OpenAI
from dotenv import load_dotenv
from utils import function_to_schema
from agents import BaseAgent, Agent, MultiAgent

### LLM Preparation 

In [22]:
# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Init Several Agents

In [23]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if a user is asking about a topic that is not handled by the activated agent.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent used for science questions named Albert.",
           handoff_description = "Call this agent if a user is asking about a science topic.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent used for music questions named Mozart.",
           handoff_description = "Call this agent if a user is asking about a music topic.")

daily_agent = Agent(base_agent = BaseAgent(client),
           name = "daily agent",
           instructions = "Agent used for daily questions named Ada.",
           handoff_description = "Call this agent if a user is asking about a daily topic.")

###  Build a Multi-Agent environment & add handoff relationships 

In [24]:
# Build a Multi-Agent environment & add handoff relationships
ma = MultiAgent(start_agent=general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent,daily_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent,daily_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent,daily_agent])
# ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent,science_agent,music_agent])



### Multi-Agent handoff example

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [25]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <agents.Agent at 0x115151910>}

In [26]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: name 'json' is not defined


### Multi-Agent dialogue example

In [10]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <agents.Agent at 0x115175ed0>,
  'message': 'why the sky is blue'},
 {'role': 'assistant',
  'content': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. Here’s how it works:\n\n1. **Sunlight Composition**: Sunlight, or white light, is made up of different colors, each with different wavelengths. Blue light has a shorter wavelength, while red light has a longer wavelength.\n\n2. **Interaction with the Atmosphere**: When sunlight enters the Earth's atmosphere, it collides with gas molecules and other small particles. Due to these collisions, shorter wavelengths of light (blue and violet) are scattered in all directions much more than longer wavelengths (red, yellow).\n\n3. **Human Perception**: Although both blue and violet light are scattered, our eyes are more sensitive to blue light. Additionally, some of the violet light is absorbed by the ozo

In [11]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, an AI trained to assist with science-related questions and inquiries. How can I help you today?',
  'agent_name': 'science_agent'}]

In [12]:
ma.chat(messages=[{"role": "user", "content": "recommend me some music"}])

[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <agents.Agent at 0x1150e77d0>,
  'message': 'recommend me some music'},
 {'role': 'handoff',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <agents.Agent at 0x115175e50>,
  'message': 'recommend me some music'},
 {'role': 'assistant',
  'content': 'Sure! What kind of music are you in the mood for? Are you looking for classical, jazz, rock, pop, or something else? Also, let me know if you have any favorite artists or styles!',
  'agent_name': 'music_agent'}]

In [13]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'handoff',
  'handoff': 'music_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <agents.Agent at 0x1150e77d0>,
  'message': 'dinner tonight'},
 {'role': 'handoff',
  'handoff': 'general_agent -> daily_agent',
  'agent_name': 'daily_agent',
  'agent': <agents.Agent at 0x1151759d0>,
  'message': 'dinner tonight'},
 {'role': 'assistant',
  'content': 'What kind of dinner are you in the mood for? I can suggest some options based on your preferences—like cuisine type, dietary restrictions, or ingredients you have on hand!',
  'agent_name': 'daily_agent'}]

In [14]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': 'I am Albert, an agent designed to assist with science questions. How can I help you today?',
  'agent_name': 'science_agent'}]

In [15]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: name 'json' is not defined


### Multi-Agent tool use example

In [16]:
def get_weather(city:str)->str:
    """ 
    Get the weather for a specified city.
    """
    return f"The weather in {city} is sunny."


def get_news()->str:
    """ 
    Get the latest news.
    """
    return "The latest news is that the sun is shining."

In [17]:
daily_agent.add_tools([get_weather,get_news])

In [18]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou?"}])

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: name 'json' is not defined
ERROR:root:Error during chat with agent.
ERROR:root:Exception: 'NoneType' object is not iterable


In [19]:
ma.chat(messages=[{"role": "user", "content": "what happened today?"}])

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: name 'json' is not defined


In [20]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou and what happened today?"}])

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: name 'json' is not defined
